In [ ]:
import digitalhub as dh

### 1. Initialize the project

In [ ]:
project = dh.get_or_create_project("AreaVerde")

### 2. Initialize project source data

Initialize the sources making a snapshot from the shared data (spire 1h, 5min, accur 1h, accur 5min)

In [ ]:
import boto3
import os


s3 = boto3.client('s3',
                    endpoint_url=os.environ.get('S3_ENDPOINT_URL'),
                    aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
                    aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'))

for year in ["2023", "2024", "2025"]:
    s3.download_file('dataspace', f'mobility-data/spire-1h-{year}/latest/spire-1h-{year}.parquet', f'spire-1h-{year}.parquet')
    project.log_dataitem(name=f"spire_flow1h_{year}_source", kind="table", source=f'spire-1h-{year}.parquet')    
    s3.download_file('dataspace', f'mobility-data/spire-accur-1h-{year}/latest/spire-accur-1h-{year}.parquet', f'spire-accur-1h-{year}.parquet')
    project.log_dataitem(name=f"spire_accur_flow1h_{year}_source", kind="table", source=f'spire-accur-1h-{year}.parquet')    
    s3.download_file('dataspace', f'mobility-data/trafic-spire/{year}/trafic-spire.parquet', f'trafic-spire.parquet')
    project.log_dataitem(name=f"spire_flow5m_{year}", kind="table", source=f'trafic-spire.parquet')    
    s3.download_file('dataspace', f'mobility-data/trafic-spire-accur/{year}/trafic-spire.parquet', f'trafic-spire-accur.parquet')
    project.log_dataitem(name=f"spire_flow5m_accur_{year}", kind="table", source=f'trafic-spire-accur.parquet')    
    

#### 2.1 Spire data

In [ ]:
try:
    func = project.get_function("convert-spira")
except:
    func = project.new_function(
                         name="convert-spira",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="../src/convert.py",
                         handler="convert_spira")

In [ ]:
for year in ["2023", "2024", "2025"]:
    di = project.get_dataitem(f"spire_flow1h_{year}_source")
    run = func.run(action="job", inputs={"spira_di": di.key}, parameters={"name": f"spira_flow_data_{year}"}, local_execution=True)

#### 2.2 Spire accuracy data

In [ ]:
try:
    func = project.get_function("convert-spira-accur")
except:
    func = project.new_function(
                         name="convert-spira-accur",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="../src/convert.py",
                         handler="convert_spira_accur")    


In [ ]:
for year in ["2023", "2024", "2025"]:
    di = project.get_dataitem(f"spire_accur_flow1h_{year}_source")
    run = func.run(action="job", inputs={"spira_accur_di": di.key}, parameters={"name": f"spira_accur_data_{year}"}, local_execution=True)

#### 2.3 Gate data (from restricted area)

In [ ]:
# try:
#     func = project.get_function("convert-gates")
# except:
func = project.new_function(
                         name="convert-gates",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="../src/convert.py",
                         handler="create_gate_data")

In [ ]:
gates_file = "AreaVerde/AREA VERDE.xlsx"
data_files=[
    "AreaVerde/Export_11_02_2025, 20_11_23 - 20_05_24.xlsx", 
    "AreaVerde/Export_11_02_2025, 20_04_24 - 20_11_24.xlsx", 
    "AreaVerde/Export_11_02_2025, 20_11_24 - 11_02_25.xlsx"]

In [ ]:
%pip install openpyxl

In [ ]:
import os
os.environ["PROTECTED_ACCESS_KEY_ID"] = ""
os.environ["PROTECTED_SECRET_ACCESS_KEY"] = ""
run = func.run(action="job", parameters={ "bucket": "protected", "gates_file": gates_file,  "data_files": data_files}, local_execution=True)